In [1]:
!pip install SentencePiece
from transformers import MarianMTModel, MarianTokenizer
import torch
class TranslateMain:
    def __init__(self, model_name_fwd='Helsinki-NLP/opus-mt-en-es', model_name_reverse='Helsinki-NLP/opus-mt-es-en'):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_fwd = MarianMTModel.from_pretrained(model_name_fwd)
        self.tokenizer_fwd = MarianTokenizer.from_pretrained(model_name_fwd)
        self.model_reverse = MarianMTModel.from_pretrained(model_name_reverse)
        self.tokenizer_reverse = MarianTokenizer.from_pretrained(model_name_reverse)

    def translate_pass(self, texts, pass_fwd_or_rev):
        if pass_fwd_or_rev == 'fwd':
            tokenizer = self.tokenizer_fwd
            model = self.model_fwd.to(self.device)
        else:
            tokenizer = self.tokenizer_reverse
            model = self.model_reverse.to(self.device)
        # Tokenize input texts
        input_ids = tokenizer.batch_encode_plus(texts, return_tensors='pt', padding=True, truncation=True)['input_ids']

        input_ids = input_ids.to(self.device)
        # Perform translation
        output_ids = model.generate(input_ids)

        # output_ids = output_ids.to('cpu')

        # Decode the translated texts
        translated_texts = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

        input_ids.to('cpu')
        return translated_texts

    def rephrase_using_translation(self, texts):
        fwd_pass_text = self.translate_pass(texts, pass_fwd_or_rev='fwd')
        print(f'Intermediate translation: {fwd_pass_text}')
        rev_pass_text = self.translate_pass(fwd_pass_text, pass_fwd_or_rev='rev')
        return fwd_pass_text, rev_pass_text

# # Example usage
# translator = TranslateMain()

# texts_to_translate = ["Hello, how are you?", "@PhotOle, we also walked down to the blockade tonight for dinner and saw it, but were unaware of the situation."]
# rephrased_texts = translator.rephrase_using_translation(texts_to_translate)

# for original_text, rephrased_text in zip(texts_to_translate, rephrased_texts):
#     print(f"Original: {original_text}")
#     print(f"Rephrased: {rephrased_text}")
#     print()


In [2]:
import json

# Opening JSON file
f = open('/content/train_questions.json')

# returns JSON object as
# a dictionary
data = json.load(f)

In [3]:
def get_remaining_string(original_string, substring):
    index = original_string.find(substring)

    if index != -1:
        # If the substring is found, return the part of the string after the substring
        remaining_string = original_string[index + len(substring):]
        return remaining_string
    else:
        # If the substring is not found, return the original string
        return original_string

substring = "Paraphrase the following tweet without any explanation before or after it:"
only_the_tweet = []
for i in data:
    only_the_tweet.append(get_remaining_string(i['input'], substring))

In [4]:
len(only_the_tweet)

10437

In [5]:
# original_text = only_the_tweet[100:200]

In [14]:
from tqdm import tqdm
import pandas as pd
# len(original_text)
def split_list(input_list, chunk_size):
    return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]

# Example usage
original_list = only_the_tweet  # Replace this with your actual list
chunk_size = 50

result = split_list(original_list, chunk_size)
checkpoint = -1
# Print the result
list_of_df = []
translator = TranslateMain()
for i, chunk in tqdm(enumerate(result), total=len(result), desc="Processing transaltion"):
    if i>checkpoint:
      torch.cuda.empty_cache()
      original_text = chunk
      rephrased_texts = translator.rephrase_using_translation(original_text)
      df = pd.DataFrame({"original text": original_text, 'fwd_translated': rephrased_texts[0], 'rev_translated': rephrased_texts[1]})
      torch.cuda.empty_cache()
      list_of_df.append(df)

checkpoint = 20


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Processing transaltion:   0%|          | 0/209 [00:00<?, ?it/s]

Intermediate translation: ['Actualmente estoy disfrutando del álbum "Escucha a Eason Chan".', 'Visite el blog para obtener la mejor cita del día, que es "cariño mi cara, puta" jajaja.', '@jackfaulkner sugirió usar heroína para ayudar a aliviar el dolor físico.', '@WillMyDogHateMe, tu declaración sobre que la teoría es poderosa y causa que la gente sienta miedo es exacta.', '@delossantoss está de acuerdo contigo @raingin. ¿Cómo estuvo tu trabajo? :o.', '@MissPressa y yo teníamos la misma idea. Desafortunadamente, ella no asistió a SMCSYD. Me di cuenta de su experiencia a través de Twitter.', '@MINGOENT, ¡es una gran noticia! Mis hijos lo están haciendo bien. La escuela está casi terminada, pero no estoy listo para ello.', 'El último espectáculo de Primavera de Despertar en Londres ha comenzado recientemente y desafortunadamente, nunca tuve la oportunidad de asistir.', 'Mi calentador se encuentra debajo de mi mesa; un buen pensamiento.', '@Dangerous_T, ¡eres increíblemente tonto! Espero 

Processing transaltion: 100%|██████████| 209/209 [00:03<00:00, 62.82it/s]


In [12]:
# pd.concate(list_of_df, axis=0)
# result = pd.concat(list_of_df, axis=0, ignore_index=True)

In [19]:
result2

,original text,fwd_translated,rev_translated
0,@ChiefEditor4SAU is joking about knowing who ...,@ChiefEditor4SAU está bromeando sobre saber a ...,@ChiefEditor4SAU is joking about knowing who t...
1,"@tprolificjones, awesome to have caught up wi...","@tprolificjones, increíble haberte alcanzado. ...","@tprolificjones, incredible to have reached yo..."
2,@diginux is almost reaching the 2-mile mark a...,@diginux está casi alcanzando la marca de 2 mi...,@diginux is almost reaching the 2-mile mark an...
3,@matt303 @pickoo Just ask if you want some.,@matt303 @pickoo Solo pregunta si quieres un p...,@matt303 @pickoo Just ask if you want some.
4,"""Anyone know why my new background isn't show...","""¿Alguien sabe por qué mi nuevo historial no a...","""Does anyone know why my new record doesn't sh..."
...,...,...,...
10382,Finally on the move after being stuck in traf...,Finalmente en movimiento después de estar atas...,Finally moving after being stuck in traffic. I...
10383,"""I can't wait for the sun to set, it's so dam...","""No puedo esperar a que se ponga el sol, hace ...","""I can't wait for the sun to set, it's so hot ..."
10384,@pat is laughing out loud and agrees with som...,@pat se está riendo en voz alta y está de acue...,@pat is laughing out loud and agrees with some...
10385,I'm thrilled! It's going to be a nail-biter. ...,Va a ser un mordedor de uñas. #TeamAidan,It's gonna be a nail biter. #TeamAidan


In [20]:
result = pd.concat([df, result2], axis=0, ignore_index=True)

In [21]:
result

,original text,fwd_translated,rev_translated
0,"I'm currently enjoying the album ""Listen to E...","Actualmente estoy disfrutando del álbum ""Escuc...","I am currently enjoying the album ""Listen to E..."
1,"Visit the blog for the best quote of the day,...",Visite el blog para obtener la mejor cita del ...,Visit the blog to get the best date of the day...
2,@jackfaulkner suggested using heroin to help ...,@jackfaulkner sugirió usar heroína para ayudar...,@jackfaulkner suggested using heroin to help r...
3,"@WillMyDogHateMe, your statement about the th...","@WillMyDogHateMe, tu declaración sobre que la ...","@WillMyDogHateMe, your statement that theory i..."
4,@delossantoss agrees with you @raingin. How w...,@delossantoss está de acuerdo contigo @raingin...,@delossantoss agrees with you @raingin. How wa...
...,...,...,...
10432,Finally on the move after being stuck in traf...,Finalmente en movimiento después de estar atas...,Finally moving after being stuck in traffic. I...
10433,"""I can't wait for the sun to set, it's so dam...","""No puedo esperar a que se ponga el sol, hace ...","""I can't wait for the sun to set, it's so hot ..."
10434,@pat is laughing out loud and agrees with som...,@pat se está riendo en voz alta y está de acue...,@pat is laughing out loud and agrees with some...
10435,I'm thrilled! It's going to be a nail-biter. ...,Va a ser un mordedor de uñas. #TeamAidan,It's gonna be a nail biter. #TeamAidan


In [22]:
result.to_csv("chunk_10437_en_es_en.csv")

In [9]:
# len(result[10])

KeyError: ignored

In [ ]:
torch.cuda.empty_cache()


In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame({"original text": original_text, 'fwd_translated': rephrased_texts[0], 'rev_translated': rephrased_texts[1]})